In [1]:
import logging
import json
import os
import re
from pathlib import Path
from collections import Counter

import anndata
import pandas as pd
import numpy as np
import tifffile
from einops import rearrange
from skimage.measure import regionprops

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import multiplex_imaging_pipeline.utils as utils
import multiplex_imaging_pipeline.ome as ome
import multiplex_imaging_pipeline.segmentation as seg
import multiplex_imaging_pipeline.multiplex_imaging_pipeline as mip
from multiplex_imaging_pipeline.spatial_features import DEFAULT_GATING_STRATEGY

2024-11-03 19:09:46.301121: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/lib:/usr/lib64:/usr/local/lib:/usr/lib64:
2024-11-03 19:09:46.301156: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# multiplex_mapping = {
#     'keratin-5': 'ck5',
#     'keratin-14': 'ck14',
# }
# pattern = r'[ -]?\(?[dD]\)?$'
# def alter_name(x):
#     x = re.sub(pattern, '', x)
#     x = x.lower()
#     x = x.replace(' ', '-')
#     x = multiplex_mapping.get(x, x)
#     return x

In [6]:
# case = 'HT413C1-Th1k4A1'
# runs = [
#     '2024-01-11_HT413C1_Th1k4A1',
#     '2024-01-10_HT413C1_Th1k4A1'
# ]
# output_dir = f'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/{case}'

In [5]:
# case = 'HT704B1'
# runs = [
#     '2024-07-05_HT704B1',
#     '2024-07-05_HT704B1_2',
#     '2024-07-05_HT704B1_3'
# ]
# output_dir = f'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/{case}'

In [5]:
# case = 'HT891Z1'
# runs = [
#     '2024-07-23_HT891Z1',
#     '2024-07-23_HT891Z1_2',
#     '2024-07-20_HT891Z1',
#     '2024-07-20_HT891Z1_2'
# ]
# output_dir = f'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/{case}'

In [8]:
# case = 'HT913Z1'
# runs = [
#     '2024-08-12_HT913Z1',
#     '2024-08-12_HT913Z1_2',
#     '2024-08-12_HT913Z1_3',
#     '2024-08-12_HT913Z1_4'
# ]
# output_dir = f'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/{case}'

In [9]:
metadata = pd.read_csv('/diskmnt/Projects/Users/estorrs/sandbox/specimen_tracking.tsv', sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
metadata

,run_name,omero_run_name,date,case,specimen,bbox,filepath,geojson
0,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U2,"562,14052,0,16950",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
1,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U12,"18424,31915,774,17324",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
2,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U22,"33663,47341,649,17012",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
3,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1,2024-08-12_HT913Z1,20240812,HT913Z1,HT913Z1-S2H1FP1_U41,"49777,63330,3084,19261",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
4,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,2024-08-12_HT913Z1_2,20240812,HT913Z1,HT913Z1-S2H1FP1_U51,"589,13911,404,16732",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
5,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,2024-08-12_HT913Z1_2,20240812,HT913Z1,HT913Z1-S2H1FP1_U52,"16387,29120,816,17498",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
6,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,2024-08-12_HT913Z1_2,20240812,HT913Z1,HT913Z1-S2H1FP1_U53,"32185,44848,1111,17675",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
7,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_2,2024-08-12_HT913Z1_2,20240812,HT913Z1,HT913Z1-S2H1FP1_U54,"46686,59772,2290,18618",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
8,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3,2024-08-12_HT913Z1_3,20240812,HT913Z1,HT913Z1-S2H1FP1_U62,"524,13396,1315,17507",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...
9,20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_3,2024-08-12_HT913Z1_3,20240812,HT913Z1,HT913Z1-S2H1FP1_U71,"17590,30171,1664,17565",/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...,/diskmnt/primary/CODEX/HTAN/20240812_Prostate_...


In [10]:
for fp in metadata['filepath']:
    channels = utils.get_ome_tiff_channels(fp)
#     channels = [alter_name(x) for x in channels]
    print(fp, channels)

/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U2__20240812.ome.tiff ['DAPI', 'CK14', 'LYVE1', 'CD4', 'CK5', 'STEAP4', 'CD68', 'E-Cadherin', 'SMA (D)', 'CD45(D)', 'p63', 'Pan-Cytokeratin', 'HLA-DR', 'Podoplanin', 'Vimentin', 'CD3e', 'CD8', 'CK7', 'CD44', 'Foxp3', 'CD20', 'Ki67', 'AR', 'AMACR', 'CD31']
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U12__20240812.ome.tiff ['DAPI', 'CK14', 'LYVE1', 'CD4', 'CK5', 'STEAP4', 'CD68', 'E-Cadherin', 'SMA (D)', 'CD45(D)', 'p63', 'Pan-Cytokeratin', 'HLA-DR', 'Podoplanin', 'Vimentin', 'CD3e', 'CD8', 'CK7', 'CD44', 'Foxp3', 'CD20', 'Ki67', 'AR', 'AMACR', 'CD31']
/diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U22__20240812.ome.tiff ['DAPI', 'CK14', 'LYVE1', 'CD4', 'CK5', 'STEAP4', 'CD68', 'E-Cadherin', 'SMA (D)', 'CD45(D)', 'p63', 'Pan-Cytokeratin', 'HLA-DR', 'Podoplanin', 'Vimentin', 'CD3e', 'CD8', 'CK7', 'CD44',

In [11]:
nuclei_channels = ['DAPI']
# membrane_channels = ['Pan-Cytokeratin', 'HLA-DR', 'SMA (D)', 'CD45 (D)', 'CD31', 'E-cadherin', 'CD68', 'CD3e', 'Vimentin']
membrane_channels = ['Pan-Cytokeratin', 'HLA-DR', 'SMA (D)', 'CD31', 'E-Cadherin', 'CD68', 'CD3e', 'Vimentin']
seg_dir = os.path.join(output_dir, 'segmentation')
Path(seg_dir).mkdir(parents=True, exist_ok=True)
for specimen, fp in zip(metadata['specimen'], metadata['filepath']):
    print(specimen)
    prefix = os.path.join(seg_dir, specimen)
    mip.segment_ome(fp, prefix, 25000, nuclei_channels, membrane_channels)
    
    seg_fp = prefix + '_cell_segmentation.tif'
    df, a = mip.get_spatial_features(seg_fp, fp, thresholds={})
    
    a.write_h5ad(os.path.join(output_dir, f'{specimen}_spatial_features.h5ad'))
    df.to_csv(os.path.join(output_dir, f'{specimen}_spatial_features.txt'), sep='\t', index=False)


2024-11-03 19:11:09,256 - starting segmentation for /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H1FP1_U2__20240812.ome.tiff
2024-11-03 19:11:09,258 - Checking for cached data


HT913Z1-S2H1FP1_U2


2024-11-03 19:11:09,510 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-11-03 19:11:09,511 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-11-03 19:11:09,512 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-11-03 19:11:12,270 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models
2024-11-03 19:11:12.457509: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/lib:/usr/lib64:/usr/local/lib:/usr/lib64:
2024-11-03 19:11:12.457565: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-11-03

2024-11-03 19:11:19,902 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-11-03 19:49:30,910 - finished segmentation
2024-11-03 19:49:30,914 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT913Z1/segmentation/HT913Z1-S2H1FP1_U2_nuclei_segmentation.tif
2024-11-03 19:49:32,311 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT913Z1/segmentation/HT913Z1-S2H1FP1_U2_cell_segmentation.tif
2024-11-03 19:49:35,517 - extracting /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_1/HT913Z1-S2H

HT913Z1-S2H1FP1_U12


...

2024-11-04 03:58:45,717 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-11-04 03:58:45,718 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-11-04 03:58:45,719 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-11-04 03:58:48,360 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models


2024-11-04 03:58:55,861 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-11-04 04:15:48,765 - finished segmentation
2024-11-04 04:15:48,771 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT913Z1/segmentation/HT913Z1-S2H1FP1_U121_nuclei_segmentation.tif
2024-11-04 04:15:49,353 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT913Z1/segmentation/HT913Z1-S2H1FP1_U121_cell_segmentation.tif
2024-11-04 04:15:50,141 - extracting /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4/HT913Z1

HT913Z1-S2H1FP1_U125


2024-11-04 04:17:26,863 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-11-04 04:17:26,864 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-11-04 04:17:26,865 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-11-04 04:17:29,735 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models


2024-11-04 04:17:36,847 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-11-04 04:35:36,781 - finished segmentation
2024-11-04 04:35:36,786 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT913Z1/segmentation/HT913Z1-S2H1FP1_U125_nuclei_segmentation.tif
2024-11-04 04:35:37,363 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT913Z1/segmentation/HT913Z1-S2H1FP1_U125_cell_segmentation.tif
2024-11-04 04:35:40,405 - extracting /diskmnt/primary/CODEX/HTAN/20240812_Prostate_serial_HT913Z1-S2H1FP1-CODEX_4/HT913Z1

In [12]:
output_dir

'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT913Z1'